# NIRISS AMI New Instrument Mode proposal
### Super- and Sub-stripe

V1: Anand

In [ ]:
# Imports
import os
import glob
import numpy as np
from astropy.io import fits

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # image module for image reading

# For notebook animations
from matplotlib import animation
from IPython.display import HTML
from IPython.display import display, Markdown, Latex
###display(Markdown('*some markdown* $\phi$'))
#### If you particularly want to display maths, this is more direct:
###display(Latex('\phi'))

# import base64.b64encode, the video file need to be encoded with base64 encoding.
from base64 import b64encode
from astropy.io import ascii
import imageio
# Next 2 lines for output control for clean export product (eg html) 
import warnings
warnings.filterwarnings('ignore')

mpl.rcParams['figure.dpi'] = 100
pd.set_option('display.max_columns', None)

# Frames per second for all movies
framespersecond = 2

# movies written to disk:
movies = []

%load_ext autoreload
%autoreload 2
    
from Info import Info


In [ ]:

regexstr = '/Users/anand/data/com1093/01093/jw01093*_nis_cal.fits'
flist = glob.glob(regexstr)
#print(flist[:6], '---\n', flist[-6:])

info1093 = Info(flist, hdrkeys=None)
info1093.print(ftype='hjgfjf ')


In [ ]:
class AMI:
    """
    Constants, etc.
    """
    
    sTable1 = """Table 1 
21 Baselines Formed by 7 NRM Subaperture Centers Projected with No Distortion onto JWST's Primary Mirror


Hole Pair	Length	Angle	
	m	deg	
1,2 (B4-C2)	3.492	-40.89	
1,3 (B4-B5)	2.640	60.00	
1,4 (B4-B2)	4.573	-30.00	
1,5 (B4-C1)	4.759	-13.90	
1,6 (B4-B6)	4.573	30.00	
1,7 (B4-C6)	4.759	13.90	
2,3 (C2-B5)	4.759	106.10	
2,4 (C2-B2)	1.320	0.00	
2,5 (C2-C1)	2.286	30.00	
2,6 (C2-B6)	4.759	73.90	
2,7 (C2-C6)	3.960	60.00	
3,4 (B5-B2)	5.280	-60.00	
3,5 (B5-C1)	4.759	-46.10	
3,6 (B5-B6)	2.640	0.00	
3,7 (B5-C6)	3.492	-19.11	
4,5 (B2-C1)	1.320	60.00	
4,6 (B2-B6)	4.573	90.00	
4,7 (B2-C6)	3.492	79.11	
5,6 (C1-B6)	3.492	100.89	
5,7 (C1-C6)	2.286	90.00	
6,7 (B6-C6)	1.320	-60.00	
Note. Angle is given in degrees counter-clockwise from the positive V3 axis. Nomenclature matches that of Figure 8.

    """
    table1df = read_jdoc_tablestring(sTable1)
    
    sTable2 = """Table 2 
AMI and KPI Image Distortion. Pixel scales for 5 points within the SUB80 (or NIS_AMI1) aperture


x	y	p _x	p _y	
pixels	pixels	mas/pixel	mas/pixel	
46.0	41.0	65.3249	65.7226	
0.5	0.5	65.3291	65.7431	
80.5	0.5	65.3248	65.7456	
80.5	80.5	65.3215	65.7063	
0.5	80.5	65.3254	65.7045	
Note. The first row is the reference pixel position (XSciRef, YSciRef in SIAF) and the other rows are the 4 subarray corners. Data file headers contain the 2 x 2 linear distortion CD matrix evaluated at the reference position, in equatorial coordinates. The matrix includes a small shear as well as the differing x and y pixel scales. From JWST Commissioning Program 1088, PI S. Sohn."""
    
    sTable3 = """Table 3 
Filter Properties for NIRISS AMI and KPI


Filter	lambda _pivot	Deltalambda/lambda	IWA	CPF-AMI	CPF-KPI	AMI FOV	AMI Bright Limit	
	mum	%	mas			arcsec	Vega mag	
F277W	2.78	26.3	89	0.045^*	0.23^*	0.79	7.5	
F380M	3.83	5.4	120	0.026	0.13^*	1.15	4.6	
F430M	4.29	5.0	140	0.022	0.11^*	1.28	4.0	
F480M	4.82	6.3	150	0.018	0.08	1.44	3.6	
Note. lambda _pivot is defined in Tokunaga & Vacca (2005). Deltalambda: half-power filter width. IWA: Inner Working Angle. CPF: Central Pixel Fraction. CPFs from simulations (starred) may be ~10% high. AMI FOV: interferometric field of view, i.e., the primary beam core's diameter. KPI's bright limit is ~4.5 mag fainter than AMI's.
References: Values found in updated JWST documentation should be used if they differ from the above. Early results are presented here.
"""

    
    # https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-operations/niriss-dithers/niriss-ami-dithers
    
    sTablePos = """Table 1. Offsets for primary AMI dithers

PrimaryDither	ΔX (arcsec)	ΔY (arcsec)
1	0.0000	0.0000
2	-1.8309	1.1271
3	-1.8307	-1.0428
4	0.9155	-1.1224
"""
    
    sTableOff = """Table 3. Offsets for a 5-step subpixel dither pattern with AMI

  Subpixel Positions = 5
Dither Step #	ΔX (arcsec)	ΔY (arcsec)
1	0.0000	0.0000
2	0.01633	0.01636
3	-0.01633	0.01650
4	-0.01633	-0.01636
5	0.01633	-0.01650"""
    
    sTable9pt = """Table 4. Offsets for a 9-step subpixel dither pattern with AMI

  Subpixel Positions = 9
Dither Step #	ΔX (arcsec)	ΔY (arcsec)
1	0.0000	0.0000
2	0.0216	0.0000
3	0.0216	0.0216
4	0.0000	0.0217
5	-0.0216	0.0218
6	-0.0216	0.0001
7	-0.0216	-0.0216
8	0.0000	-0.0217
9	0.0216	-0.0218"""
    
    sTable25pt = """Table 5. Offsets for a 25-step subpixel dither pattern with AMI

Subpixel Positions = 25
Dither Step #	ΔX (arcsec)	ΔY (arcsec)
1	0.0000	0.0000
2	0.0131	-0.0001
3	0.0262	-0.0001
4	0.0262	0.0130
5	0.0262	0.0262
6	0.0131	0.0262
7	0.0000	0.0263
8	-0.0131	0.0264
9	-0.0262	0.0264
10	-0.0262	0.0133
11	-0.0262	0.0000
12	-0.0262	-0.0130
13	-0.0262	-0.0262
14	-0.0131	-0.0262
15	0.0000	-0.0263
16	0.0131	-0.0264
17	0.0262	-0.0264
18	0.0262	-0.0133
19	0.0131	-0.0132
20	0.0000	-0.0132
21	-0.0131	-0.0131
22	-0.0131	0.0001
23	-0.0131	0.0132
24	0.0000	0.0132
25	0.0131	0.0131"""

In [ ]:
#Hi Anand, Here is one way of making the table. If you are using calints file you can use ImageModel instead of RampModel.

from jwst.datamodels import RampModel
datadir = 'pipeline_calibrated_data'
rampfiles = sorted(glob.glob(os.path.join(datadir,'jw*ramp.fits')))

for df in rampfiles:
    ramp = RampModel(df)
    print(ramp.meta.filename,"\t", ramp.meta.observation.observation_number, "\t", ramp.meta.target.proposer_name,"\t", \
          ramp.meta.exposure.type,"\t", ramp.meta.aperture.name, "\t", ramp.meta.instrument.pupil,"\t", ramp.meta.instrument.filter,"\t",\
          ramp.meta.subarray.name,"\t", ramp.meta.exposure.nints, "\t", ramp.meta.exposure.ngroups, "\t", \
          ramp.meta.dither.position_number, "\t", ramp.meta.dither.total_points)